In [1]:
import os
from pathlib import Path

while Path.cwd().name.lower() != "aiml25-exam" and "aiml25-exam" in str(Path.cwd()).lower():
    os.chdir("..")  # Move up one directory
print(f"Working directory set to: {Path.cwd()}")

Working directory set to: /Users/emilstausbol/Documents/GitHub/AIML25-Exam


In [2]:
from src.yolo.yolo import Yolo
from ultralytics import YOLO
from src.utils.path import from_root
from src.llm_caller import LLMCaller
from src.llm_detector import Detector
from src.mermaid_detector import MermaidDetector
from src.mermaid_to_json import MermaidToJSON
from src.edge_validator import EdgeValidator
import matplotlib.pyplot as plt
import json
plt.figure(figsize=(14, 10))

<Figure size 1400x1000 with 0 Axes>

<Figure size 1400x1000 with 0 Axes>

In [3]:
model = YOLO(from_root("models/yolo-trained.pt"))

yolo = Yolo(model)

In [4]:
from decouple import config

# WX_API_KEY = config("WX_API_KEY")
# WX_PROJECT_ID = config("WX_PROJECT_ID")
# WX_API_URL = "https://us-south.ml.cloud.ibm.com"
OPENAI_KEY = config("OPENAI_API_KEY")
# OPENAI_API_URL = "https://api.openai.com/v1"
# OPENAI_PROJECT = "proj_6SmPVkyXa0GChFv7TmDjB9B5"

model = LLMCaller(
    api_key=OPENAI_KEY,          
    model_id="gpt-4o",     
    params={
        "temperature": 0.7,
        "max_tokens": 150
    })

In [5]:
detector = Detector(model, yolo)
mermaidDetector = MermaidDetector(model, yolo, MermaidToJSON(model))

In [6]:
def mermaid_with_nodes_crop(path: str, json_path: str):
    mermaidDetector.initiate_image(str(from_root(path)))
    mermaidDetector.detect_nodes()
    mermaidDetector.detect_edges()
    mermaidDetector.convert_edges()
    graph = mermaidDetector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def mermaid_crop(path: str, json_path: str):
    mermaidDetector.initiate_image(str(from_root(path)))
    mermaidDetector.detect_edges()
    mermaidDetector.convert_edges()
    graph = mermaidDetector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def mermaid_no_crop(path: str, json_path: str):
    mermaidDetector.initiate_image(str(from_root(path)), should_crop=False)
    mermaidDetector.detect_edges()
    mermaidDetector.convert_edges()
    graph = mermaidDetector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def mermaid_with_nodes_no_crop(path: str, json_path: str):
    mermaidDetector.initiate_image(str(from_root(path)), should_crop=False)
    mermaidDetector.detect_nodes()
    mermaidDetector.detect_edges()
    mermaidDetector.convert_edges()
    graph = mermaidDetector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def with_nodes_crop(path: str, json_path: str):
    detector.initiate_image(str(from_root(path)))
    detector.detect_nodes()
    detector.detect_edges()
    graph = detector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def crop(path: str, json_path: str):
    detector.initiate_image(str(from_root(path)))
    detector.detect_edges()
    graph = detector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def no_crop(path: str, json_path: str):
    detector.initiate_image(str(from_root(path)), should_crop=False)
    detector.detect_edges()
    graph = detector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

def with_nodes_no_crop(path: str, json_path: str):
    detector.initiate_image(str(from_root(path)), should_crop=False)
    detector.detect_nodes()
    detector.detect_edges()
    graph = detector.get_graph()
    validator = EdgeValidator.from_json_file(str(from_root(json_path)), graph.edges)
    return validator.validate()

In [7]:
dataset = [
    [
        "datasets/test/images/1.png",
        "datasets/test/json/1.json",
    ],
    [
        "datasets/test/images/3.png",
        "datasets/test/json/3.json",
    ],
    [
        "datasets/test/images/4.png",
        "datasets/test/json/4.json",
    ],
    [
        "datasets/test/images/5.png",
        "datasets/test/json/5.json",
    ],
    [
        "datasets/test/images/6.png",
        "datasets/test/json/6.json",
    ],
    [
        "datasets/test/images/7.png",
        "datasets/test/json/7.json",
    ],
    [
        "datasets/test/images/8.png",
        "datasets/test/json/8.json",
    ],
    [
        "datasets/test/images/9.png",
        "datasets/test/json/9.json",
    ],
    [
        "datasets/test/images/10.png",
        "datasets/test/json/10.json",
    ],
]

In [8]:
scores = {
    "mermaid_no_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "mermaid_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "mermaid_with_nodes_no_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "mermaid_with_nodes_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "no_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "with_nodes_no_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    },
    "with_nodes_crop": {
        "precision": [],
        "recall": [],
        "f1_score": []
    }
}


In [9]:
def appendScores(val_scores: dict, fn: str):
    s = scores[fn]
    precision = val_scores["precision"]
    s["precision"].append(precision)
    f1_score = val_scores["f1_score"]
    s["f1_score"].append(f1_score)
    recall = val_scores["recall"]
    s["recall"].append(recall)

In [ ]:
# Isaac
for [img_path, json_path] in dataset:
    appendScores(mermaid_crop(img_path, json_path), "mermaid_crop")
    appendScores(mermaid_with_nodes_crop(img_path, json_path), "mermaid_with_nodes_crop")
    appendScores(mermaid_no_crop(img_path, json_path), "mermaid_no_crop")

with open('isaac.json', 'w') as f:
    json.dump(scores, f, indent=4)

In [10]:
# Emil
for [img_path, json_path] in dataset:
    appendScores(mermaid_with_nodes_no_crop(img_path, json_path), "mermaid_with_nodes_no_crop")
    appendScores(crop(img_path, json_path), "crop")
    appendScores(with_nodes_crop(img_path, json_path), "with_nodes_crop")

with open('emil.json', 'w') as f:
    json.dump(scores, f, indent=4)

/opt/anaconda3/envs/aiml25-exam/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/aiml25-exam/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /opt/anaconda3/envs/aiml25-exam/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/aiml25-exam/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/aiml25-exam/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/aiml25-exam/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/aiml25-exam/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your e

image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/datasets/test/images/1.png: 448x640 14 sub-flows, 1202.6ms
Speed: 5.7ms preprocess, 1202.6ms inference, 11.1ms postprocess per image at shape (1, 3, 448, 640)
totale images: 14

image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/datasets/test/images/1.png: 448x640 14 sub-flows, 1148.5ms
Speed: 7.0ms preprocess, 1148.5ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)
totale images: 14

image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/datasets/test/images/3.png: 224x640 8 sub-flows, 639.6ms
Speed: 35.1ms preprocess, 639.6ms inference, 13.7ms postprocess per image at shape (1, 3, 224, 640)
totale images: 8

image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/datasets/test/images/3.png: 224x640 8 sub-flows, 607.8ms
Speed: 2.3ms preprocess, 607.8ms inference, 1.1ms postprocess per image at shape (1, 3, 224, 640)
totale images: 8

image 1/1 /Users/emilstausbol/Documents/GitHub/AIML25-Exam/da

In [11]:
# Simon
for [img_path, json_path] in dataset:
    appendScores(no_crop(img_path, json_path), "no_crop")
    appendScores(with_nodes_no_crop(img_path, json_path), "with_nodes_no_crop")

with open('simon.json', 'w') as f:
    json.dump(scores, f, indent=4)